In [ ]:
# import packages
import numpy as np
import pandas as pd
import pyreadstat
import random
import networkx as nx
import matplotlib.pyplot as plt
from pgmpy.inference import DBNInference
from pgmpy.models import DynamicBayesianNetwork as DBN
from sklearn.model_selection import train_test_split 
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# set random seed
random.seed(9)

In [ ]:
# create model
model = DBN()

In [ ]:
# add all edges
model.add_edges_from([
    (('Age', 0), ('Recurrence', 0)),
    (('Sex', 0), ('Recurrence', 0)),
    (('Stage', 0), ('Recurrence', 0)),
    (('Stage', 0), ('Treatment_change', 0)),
    (('Treatment_change', 0), ('Recurrence', 0)),
    (('Histology', 0), ('Recurrence', 0)),
    (('Other_conditions', 0), ('Recurrence', 0)),

    (('Age', 1), ('Recurrence', 1)),
    (('Sex', 1), ('Recurrence', 1)),
    (('Stage', 1), ('Recurrence', 1)),
    (('Stage', 1), ('Treatment_change', 1)),
    (('Treatment_change', 1), ('Recurrence', 1)),
    (('Histology', 1), ('Recurrence', 1)),
    (('Other_conditions', 1), ('Recurrence', 1)),

    (('Age', 2), ('Recurrence', 2)),
    (('Sex', 2), ('Recurrence', 2)),
    (('Stage', 2), ('Recurrence', 2)),
    (('Stage', 2), ('Treatment_change', 2)),
    (('Treatment_change', 2), ('Recurrence', 2)),
    (('Histology', 2), ('Recurrence', 2)),
    (('Other_conditions', 2), ('Recurrence', 2)),
    
    (('Recurrence', 0), ('Recurrence', 1)),
    (('Recurrence', 1), ('Recurrence', 2))
])

In [ ]:
# get model graph
graph = nx.MultiGraph()
graph.add_edges_from(model.edges())

# plot model
pos = nx.spring_layout(graph)
nx.draw(graph, pos, with_labels=True, node_size=3000, node_color='lightblue', font_size=5, font_weight='bold')
plt.title("Model Structure")
plt.show()

In [ ]:
# load data
ziekte_data = pd.read_excel("ziekte_lang.xlsx")
sympro_data, meta = pyreadstat.read_sav("20230321_SYMPRO.sav")
behandeling_data = pd.read_excel("behandeling_lang.xlsx")

In [ ]:
# choose columns for the model
treatment_data = behandeling_data.drop(["event_key", "behandeling", "startdatum_behandeling", "stopdatum_behandeling", 
                                        "behandeling_categorie", "epis_nr"], axis = 1)
treatment_data = treatment_data.groupby(["sympro_respondent", "startdatum_behandelcombinatie", 
                                         "stopdatum_behandelcombinatie"]).first().reset_index()

symptoms_data = sympro_data.loc[:, ["respondent", "bas_dem_age", "bas_dem_gender", "t0crf_klinv1", "T0_comb"]]
symptoms_data = symptoms_data.rename(columns={"respondent": "sympro_respondent"})

recurrence_data = ziekte_data.loc[:, ["sympro_respondent", "event_date", "stadium_tnm7_8", "recurrence"]]

# encode age into bins
bins = [18, 44, 54, 65, 100]
labels = ['under 45', '45-54', '55-65', 'over 65']
symptoms_data["bas_dem_age"] = pd.cut(symptoms_data["bas_dem_age"], bins=bins, labels=labels, right=True, ordered=True)

In [ ]:
# merge datasets
merged_data = pd.merge(treatment_data, recurrence_data, on='sympro_respondent', how='inner')
merged_data = pd.merge(merged_data, symptoms_data, on='sympro_respondent', how='inner')

# t0 - three months
def after_three_months(participant):
    start_date = participant['event_date'].min()
    cutoff = start_date + pd.DateOffset(months=3)
    filtered = participant[participant['event_date'] < cutoff]
    filtered = filtered[filtered['startdatum_behandelcombinatie'] < cutoff]
    return filtered

merged_data_t0 = merged_data.groupby("sympro_respondent", group_keys=False).apply(after_three_months)
merged_data_t0 = merged_data_t0.groupby('sympro_respondent').agg(lambda x: ', '.join(sorted(set(x.astype(str))))).reset_index()

# t1 - six months
def after_six_months(participant):
    start_date = participant['event_date'].min()
    cutoff = start_date + pd.DateOffset(months=6)
    filtered = participant[participant['event_date'] < cutoff]
    filtered = filtered[filtered['startdatum_behandelcombinatie'] < cutoff]
    return filtered

merged_data_t1 = merged_data.groupby("sympro_respondent", group_keys=False).apply(after_six_months)
merged_data_t1 = merged_data_t1.groupby('sympro_respondent').agg(lambda x: ', '.join(sorted(set(x.astype(str))))).reset_index()

# t2 - a year
def after_year(participant):
    start_date = participant['event_date'].min()
    cutoff = start_date + pd.DateOffset(months=13) #to include edge dates
    filtered = participant[participant['event_date'] < cutoff]
    filtered = filtered[filtered['startdatum_behandelcombinatie'] < cutoff]
    return filtered

merged_data_t2 = merged_data.groupby("sympro_respondent", group_keys=False).apply(after_year)
merged_data_t2 = merged_data_t2.groupby('sympro_respondent').agg(lambda x: ', '.join(sorted(set(x.astype(str))))).reset_index()

In [ ]:
# helper function
def get_last_valid_value(column):
    parts = [x.strip() for x in column.split(', ') if x.strip().lower() != 'nan']
    return parts[-1] if parts else 4

In [ ]:
#t0
t0 = merged_data_t0.loc[:, ["sympro_respondent"]]
t0['Age'] = merged_data_t0['bas_dem_age']
t0['Sex'] = merged_data_t0['bas_dem_gender'].apply(lambda x: float(x)).astype(int)
t0['Stage'] = merged_data_t0['stadium_tnm7_8'].apply(get_last_valid_value).astype(float).astype(int)
t0['Treatment_change'] = merged_data_t0['behandelcombi_binnen_episode'].apply(lambda x: 1 if len(str(x).split(', ')) > 1 else 0).astype(int)
t0['Histology'] = merged_data_t0['t0crf_klinv1'].apply(lambda x: float(x)).astype(int)
t0['Other_conditions'] = merged_data_t0['T0_comb'].apply(lambda x: 0.0 if x.strip().lower() == 'nan' else float(x)).astype(int)
t0['Recurrence'] = merged_data_t0['recurrence'].apply(get_last_valid_value).astype(float).astype(int)

#t1
t1 = merged_data_t1.loc[:, ["sympro_respondent"]]
t1['Age'] = merged_data_t1['bas_dem_age']
t1['Sex'] = merged_data_t1['bas_dem_gender'].apply(lambda x: float(x)).astype(int)
t1['Stage'] = merged_data_t1['stadium_tnm7_8'].apply(get_last_valid_value).astype(float).astype(int)
t1['Treatment_change'] = merged_data_t1['behandelcombi_binnen_episode'].apply(lambda x: 1 if len(str(x).split(', ')) > 1 else 0).astype(int)
t1['Histology'] = merged_data_t1['t0crf_klinv1'].apply(lambda x: float(x)).astype(int)
t1['Other_conditions'] = merged_data_t1['T0_comb'].apply(lambda x: 0.0 if x.strip().lower() == 'nan' else float(x)).astype(int)
t1['Recurrence'] = merged_data_t1['recurrence'].apply(get_last_valid_value).astype(float).astype(int)

#t2
t2 = merged_data_t2.loc[:, ["sympro_respondent"]]
t2['Age'] = merged_data_t2['bas_dem_age']
t2['Sex'] = merged_data_t2['bas_dem_gender'].apply(lambda x: float(x)).astype(int)
t2['Stage'] = merged_data_t2['stadium_tnm7_8'].apply(get_last_valid_value).astype(float).astype(int)
t2['Treatment_change'] = merged_data_t2['behandelcombi_binnen_episode'].apply(lambda x: 1 if len(str(x).split(', ')) > 1 else 0).astype(int)
t2['Histology'] = merged_data_t2['t0crf_klinv1'].apply(lambda x: float(x)).astype(int)
t2['Other_conditions'] = merged_data_t2['T0_comb'].apply(lambda x: 0.0 if x.strip().lower() == 'nan' else float(x)).astype(int)
t2['Recurrence'] = merged_data_t2['recurrence'].apply(get_last_valid_value).astype(float).astype(int)

In [ ]:
# final preprocessing
colnames = []
for t in range(3):
    colnames.extend([("Age", t), ("Sex", t), ("Stage", t), ("Treatment_change", t),
                        ("Histology", t), ("Other_conditions", t), ("Recurrence", t)])

final_data = pd.merge(t0, t1, on='sympro_respondent')
final_data = pd.merge(final_data, t2, on='sympro_respondent')
final_data = final_data.drop(columns=['sympro_respondent'])
final_data.columns = colnames
final_data.head()

In [ ]:
# check model
model.check_model()

In [ ]:
# split into train-test
train, test = train_test_split(final_data, test_size=0.3, random_state=42)

# train the model
model.fit(train)

In [ ]:
# inference example
infer = DBNInference(model)

query_result_older = infer.query(
    variables=[('Recurrence', 2)],
    evidence={('Treatment_change', 2): 1, ('Stage', 2): 4, ('Age', 2): "over 65"}
)[('Recurrence', 2)].values
query_result_younger = infer.query(
    variables=[('Recurrence', 2)],
    evidence={('Treatment_change', 2): 1, ('Stage', 2): 4, ('Age', 2): "under 45"}
)[('Recurrence', 2)].values

print(f"Recurrence_2 for an older (over 65) patient: No({query_result_older[0]}), Yes({query_result_older[1]})")
print(f"Recurrence_2 for a younger (under 45) patient: No({query_result_younger[0]}), Yes({query_result_younger[1]})")

In [ ]:
# accuracy given only t0
for j in [0,1,2]:
    correct = 0
    total = len(test)
    for i, row in test.iterrows():
        evidence={
        ('Age', 0): row[('Age', 0)],
        ('Sex', 0): row[('Sex', 0)],
        ('Stage', 0): row[('Stage', 0)],
        ('Treatment_change', 0): row[('Treatment_change', 0)],
        ('Histology', 0): row[('Histology', 0)],
        ('Other_conditions', 0): row[('Other_conditions', 0)]
        }
        result = infer.query(variables=[('Recurrence', j)], evidence=evidence)[('Recurrence', j)].values
        predicted = np.argmax(result)
        actual = row[('Recurrence', j)]
        if predicted == actual:
            correct += 1
        
    accuracy = correct / total
    print(f"Accuracy for Recurrence_{j}: {accuracy}")

In [ ]:
# accuracy given t0 and t1
for j in [0,1,2]:
    correct = 0
    total = len(test)
    for i, row in test.iterrows():
        evidence={
        ('Age', 0): row[('Age', 0)],
        ('Sex', 0): row[('Sex', 0)],
        ('Stage', 0): row[('Stage', 0)],
        ('Treatment_change', 0): row[('Treatment_change', 0)],
        ('Histology', 0): row[('Histology', 0)],
        ('Other_conditions', 0): row[('Other_conditions', 0)],
        ('Age', 1): row[('Age', 1)],
        ('Sex', 1): row[('Sex', 1)],
        ('Stage', 1): row[('Stage', 1)],
        ('Treatment_change', 1): row[('Treatment_change', 1)],
        ('Histology', 1): row[('Histology', 1)],
        ('Other_conditions', 1): row[('Other_conditions', 1)]
        }
        result = infer.query(variables=[('Recurrence', j)], evidence=evidence)[('Recurrence', j)].values
        predicted = np.argmax(result)
        actual = row[('Recurrence', j)]
        if predicted == actual:
            correct += 1
            
    accuracy = correct / total
    print(f"Accuracy for Recurrence_{j}: {accuracy}")

In [ ]:
# accuracy given all information
predicted_values = []
for j in [0,1,2]:
    correct = 0
    total = len(test)
    vals = []
    for i, row in test.iterrows():
        evidence={
        ('Age', 0): row[('Age', 0)],
        ('Sex', 0): row[('Sex', 0)],
        ('Stage', 0): row[('Stage', 0)],
        ('Treatment_change', 0): row[('Treatment_change', 0)],
        ('Histology', 0): row[('Histology', 0)],
        ('Other_conditions', 0): row[('Other_conditions', 0)],
        ('Age', 1): row[('Age', 1)],
        ('Sex', 1): row[('Sex', 1)],
        ('Stage', 1): row[('Stage', 1)],
        ('Treatment_change', 1): row[('Treatment_change', 1)],
        ('Histology', 1): row[('Histology', 1)],
        ('Other_conditions', 1): row[('Other_conditions', 1)],
        ('Age', 2): row[('Age', 2)],
        ('Sex', 2): row[('Sex', 2)],
        ('Stage', 2): row[('Stage', 2)],
        ('Treatment_change', 2): row[('Treatment_change', 2)],
        ('Histology', 2): row[('Histology', 2)],
        ('Other_conditions', 2): row[('Other_conditions', 2)]
        }
        result = infer.query(variables=[('Recurrence', j)], evidence=evidence)[('Recurrence', j)].values
        predicted = np.argmax(result)
        vals.append(predicted)
        actual = row[('Recurrence', j)]
        if predicted == actual:
            correct += 1
            
    predicted_values.append(vals)
    accuracy = correct / total
    print(f"Accuracy for Recurrence_{j}: {accuracy}")

In [ ]:
# confusion matrices for Recurrence at t0, t1 and t2 respectively
true_t0 = test[('Recurrence', 0)].tolist()
true_t1 = test[('Recurrence', 1)].tolist()
true_t2 = test[('Recurrence', 2)].tolist()

ConfusionMatrixDisplay.from_predictions(true_t0, predicted_values[0])
ConfusionMatrixDisplay.from_predictions(true_t1, predicted_values[1])
ConfusionMatrixDisplay.from_predictions(true_t2, predicted_values[2])

In [ ]:
# cpd for age_0
cpd = model.get_cpds(('Age', 0))
print("CPD for Age_0:")
print(cpd)
# cpd for stage_0
cpd = model.get_cpds(('Stage', 0))
print("CPD for Stage_0:")
print(cpd)
# cpd for stage_1
cpd = model.get_cpds(('Stage', 1))
print("CPD for Stage_1:")
print(cpd)
#cpd for treatment_change_0
cpd = model.get_cpds(('Treatment_change', 0)).to_dataframe()
print("CPD for Treatment_change_0:")
cpd.head(4)

In [ ]:
#cpd for treatment_change_1
cpd = model.get_cpds(('Treatment_change', 1)).to_dataframe()
print("CPD for Treatment_change_1:")
cpd.head(4)

In [ ]:
#cpd for recurrence_1
cpd = model.get_cpds(('Recurrence', 1)).to_dataframe()
print("CPD for Recurrence_1:")
cpd.head(100)